### 解析sql

In [1]:
from src.utils import extract_tables_and_columns, show

# 示例
sqls = [
    "SELECT FirstIndustryName FROM astockindustrydb.lc_exgindchange AS ind_change WHERE CompanyCode = 4090 AND Standard = 41 AND DATE(InfoPublDate) <= '2021-12-31' ORDER BY InfoPublDate DESC LIMIT 1;"
]

for sql in sqls:
    result = extract_tables_and_columns(sql)
    show(result)

{
  "table_to_columns": {
    "astockindustrydb.lc_exgindchange": []
  },
  "unassigned_columns": [
    "FirstIndustryName",
    "InfoPublDate",
    "CompanyCode",
    "Standard",
    "InfoPublDate"
  ]
}


### 提取sql

In [2]:
from src.utils import extract_all_sqls, show

text = '''
【已知信息】
- 天士力的关联信息有：所在数据表是constantdb.secumain; InnerCode(证券内部编码)是1700; CompanyCode(公司代码)是1474; ChiName(中文名称)是天士力医药集团股份有限公司; SecuCode(证券代码)是600535; ChiNameAbbr(中文名称缩写)是天士力。
- 涉及的表：astockeventsdb.lc_warrant（公司担保明细）

【用户的问题】
1. 天士力在2020年的最大担保金额是多少？（答案需要包含1位小数）
2. 天士力在2020年的最大担保金额涉及的担保方是谁？（请回答公司全称）
3. 担保金额是多少？（答案需要包含2位小数）
4. 天士力在2020年最新的担保事件是什么？（答案包括事件内容、担保方（公司全称）、被担保方（公司全称）、担保金额（答案需要包含1位小数）和日期信息（格式为YYYY年MM月DD日））

【思维链】
1. **最大担保金额**：
   - 需要从astockeventsdb.lc_warrant表中获取天士力在2020年的担保记录。
   - 使用CompanyCode字段筛选天士力的记录。
   - 使用InitialInfoPublDate字段筛选2020年的记录。
   - 使用MAX函数获取最大担保金额LatestGuaranteeSum，并保留1位小数。

2. **最大担保金额涉及的担保方**：
   - 在获取最大担保金额的基础上，进一步获取该记录的担保方GuarantorCompany。

3. **最新担保事件**：
   - 使用InitialInfoPublDate字段筛选2020年的记录，并按该字段降序排序。
   - 获取第一条记录，包含事件内容EventContent、担保方GuarantorCompany、被担保方SecuredParty、担保金额LatestGuaranteeSum（保留1位小数）和日期信息InitialInfoPublDate（格式化为YYYY年MM月DD日）。

【执行SQL语句】
首先，获取天士力在2020年的最大担保金额。

```exec_sql
SELECT MAX(LatestGuaranteeSum) AS A
FROM astockeventsdb.lc_warrant
WHERE CompanyCode = 1474 AND YEAR(InitialInfoPublDate) = 2020;

SELECT MAX(LatestGuaranteeSum) AS B
FROM astockeventsdb.lc_warrant
WHERE CompanyCode = 1474 AND YEAR(InitialInfoPublDate) = 2020;
```

alakaka


```exec_sql
SELECT MAX(LatestGuaranteeSum) AS C
FROM astockeventsdb.lc_warrant
WHERE CompanyCode = 1474 AND YEAR(InitialInfoPublDate) = 2020;
```

接下来，我们将分步获取其他所需信息。

```sql
SELECT MAX(LatestGuaranteeSum) AS D
FROM astockeventsdb.lc_warrant
WHERE CompanyCode = 1474 AND YEAR(InitialInfoPublDate) = 2020;
```
dkdkalala

```sql
SELECT MAX(LatestGuaranteeSum) AS E
FROM astockeventsdb.lc_warrant
WHERE CompanyCode = 1474 AND YEAR(InitialInfoPublDate) = 2020;
SELECT MAX(LatestGuaranteeSum) AS F
FROM astockeventsdb.lc_warrant
WHERE CompanyCode = 1474 AND YEAR(InitialInfoPublDate) = 2020;
```

'''

sqls = extract_all_sqls(text, "exec_sql")
for sql in sqls:
    print(sql)
    print("-"*100)


SELECT MAX(LatestGuaranteeSum) AS A
FROM astockeventsdb.lc_warrant
WHERE CompanyCode = 1474 AND YEAR(InitialInfoPublDate) = 2020;
----------------------------------------------------------------------------------------------------
SELECT MAX(LatestGuaranteeSum) AS B
FROM astockeventsdb.lc_warrant
WHERE CompanyCode = 1474 AND YEAR(InitialInfoPublDate) = 2020;
----------------------------------------------------------------------------------------------------
SELECT MAX(LatestGuaranteeSum) AS C
FROM astockeventsdb.lc_warrant
WHERE CompanyCode = 1474 AND YEAR(InitialInfoPublDate) = 2020;
----------------------------------------------------------------------------------------------------


### 重写问题

In [1]:
import os, copy, config, agents
from src.utils import show
import utils
ag_rewrite_question = agents.agent_rewrite_question.clone()
os.environ['DEBUG'] = '1'
os.environ['SHOW_LLM_INPUT_MSG'] = '1'
t_idx = 0
q_idx = 2
question_team = config.all_question[t_idx]
question_item = question_team["team"][q_idx]
question = utils.ajust_org_question(question_item["question"])
show(question_item["id"])
facts = []
qas = []
sql_results = []
for idx in range(q_idx):
    qas.append([
        {"role": "user", "content": utils.ajust_org_question(question_team["team"][idx]["question"])},
        {"role": "assistant", "content": question_team["team"][idx]["answer"]},
    ])
    if "sql_results" in question_team["team"][idx]:
        sql_results.append(copy.deepcopy(question_team["team"][idx]["sql_results"]))
    else:
        sql_results.append([])
if "facts" in question_item:
    facts = copy.deepcopy(question_item["facts"])

# rewrite question
qas_content = [
    f"Question: {qa[0]['content']}\nAnswer: {qa[1]['content']}"
    for qa in qas
]
new_question, _ = ag_rewrite_question.answer(
    ("历史问答:无。\n" if len(qas_content) == 0 else "下面是顺序的历史问答:\n'''\n" + "\n".join(qas_content) + "\n'''\n") +
    "现在用户继续提问，请根据已知信息，理解当前这个问题的完整含义，并重写这个问题使得单独拿出来看仍然能够正确理解。用户的问题是：\n" +
    question
)

TableGraph: 已从 /data/workspace/howard/jinglever/competition/2024-FinGLM2-semi-final/cache/table_relations.json 加载表关系图
tttt----1----28-1-3


>>>>> 【user】 Said:
下面是顺序的历史问答:
'''
Question: 天士力在2020年的最大担保金额是多少？答案需要包含1位小数
Answer: 天士力在2020年的最大担保金额是1620000000.0元
Question: 天士力在2020年的最大担保金额涉及的担保方是谁？（请回答公司全称）担保金额是多少（答案需要包含2位小数）？
Answer: 担保方: 天士力医药集团股份有限公司, 金额: 1620000000.00元
'''
现在用户继续提问，请根据已知信息，理解当前这个问题的完整含义，并重写这个问题使得单独拿出来看仍然能够正确理解。用户的问题是：
天士力在2020年最新的担保事件是什么？答案包括事件内容、担保方（公司全称）、被担保方（公司全称）、担保金额（答案需要包1位小数）和日期信息（格式为YYYY年MM月DD日）。


>>>>> Agent【rewrite_question】 Said:
天士力在2020年最新的担保事件是什么？请提供事件内容、担保方（公司全称）、被担保方（公司全称）、担保金额（答案需要包含1位小数）和日期信息（格式为YYYY年MM月DD日）。


### 获取当前年份

In [1]:
import datetime
datetime.datetime.now().year

2025

### SQL查询工作流

In [1]:
import os, copy, json, config, workflows
from src.utils import show
import utils
os.environ['DEBUG'] = '1'
os.environ['SHOW_LLM_INPUT_MSG'] = '1'
wf_sql_query = workflows.sql_query.clone()
wf_check_db_structure = workflows.check_db_structure.clone()
t_idx = 19
q_idx = 2
db_info = '''
{
  "astockmarketquotesdb.qt_dailyquote": ["TradingDay", "TurnoverRate", "InnerCode"],
  "constantdb.secumain": ["InnerCode", "ChiNameAbbr"]
}
'''
column_filter = json.loads(db_info)
db_info = wf_check_db_structure.filter_column_list(
    table_columns=[],
    column_filter=column_filter,
)
show(db_info)

question_team = config.all_question[t_idx]
question_item = question_team["team"][q_idx]
show(question_item["id"])
show(question_item["rewrited_question"])
facts = []
qas = []
sql_results = []
for idx in range(q_idx):
    qas.append([
        {"role": "user", "content": utils.ajust_org_question(question_team["team"][idx]["question"])},
        {"role": "assistant", "content": question_team["team"][idx]["answer"]},
    ])
    if "sql_results" in question_team["team"][idx]:
        sql_results.append(copy.deepcopy(question_team["team"][idx]["sql_results"]))
    else:
        sql_results.append([])
if "facts" in question_item:
    facts = copy.deepcopy(question_item["facts"])
# 注入已知事实
key_facts = "已知信息"
if len(facts) > 0:
    kv = {key_facts: "\n".join(facts)}
    wf_sql_query.agent_master.add_system_prompt_kv(kv)

# 注入历史对话以及支撑它的SQL查询
key_qas = "历史对话"
if len(qas_content) > 0:
    val_qas = ""
    for qa_idx, qa_content in enumerate(qas_content):
        if qa_idx > 0:
            val_qas += "---\n"
        val_qas += f"{qa_content}\n"
        if sql_results[qa_idx] != []:
            val_qas += (
                "用到以下SQL查询（供后续问答理解本问题的答案如何得来，后续问答可参考）：\n" +
                "\n".join(sql_results[qa_idx])
            )
    kv = {key_qas: val_qas}
    wf_sql_query.agent_master.add_system_prompt_kv(kv)

res = wf_sql_query.run(inputs={"messages":[
        {"role": "assistant", "content": db_info},
        {"role": "user", "content": question_item["rewrited_question"]},
    ]})

已从 /data/workspace/howard/jinglever/competition/2024-FinGLM2-semi-final/cache/table_relations.json 加载表关系图
已取得可用的数据表的字段信息如下:
{"表名": "astockmarketquotesdb.qt_dailyquote", "表字段": {"InnerCode": "证券内部编码;备注:证券内部编码（InnerCode）：与“证券主表（constantdb.secumain）”中的“证券内部编码（InnerCode）”关联，得到证券的交易代码、简称等。;数据示例:28;", "TradingDay": "交易日;数据示例:2019-01-02 12:00:00.000;"}}
---
{"表名": "constantdb.secumain", "表字段": {"InnerCode": "证券内部编码;数据示例:4427;", "ChiNameAbbr": "中文名称缩写;数据示例:横店东磁;"}}
表之间的外链关系如下:
<astockmarketquotesdb.qt_dailyquote -- [证券内部编码 (InnerCode-InnerCode)] -- constantdb.secumain>

tttt----20----14-3-3
2019年5月31日，当日换手率最高的公司是哪一家？（请回答公司中文简称）


>>>>> 【user】 Said:
2019年5月31日，当日换手率最高的公司是哪一家？（请回答公司中文简称）


>>>>> Agent【Sql_query.master】 Said:
【已知信息】
- 我们需要查询的是2019年5月31日的数据。
- 我们需要找到当日换手率最高的公司。
- 换手率在`astockmarketquotesdb.qt_dailyquote`表中，字段名为`TurnoverRate`。
- 公司的中文简称在`constantdb.secumain`表中，字段名为`ChiNameAbbr`。
- 两表通过`InnerCode`字段关联。

【用户的问题】
2019年5月31日，当日换手率最高的公司是哪一家？（请回答公司中文简称）

【思维链】
1. 首先，我们需要在`astockmarketquot

KeyboardInterrupt: 

### DB召回

In [2]:
import os, copy, config, workflows
from src.utils import show
import utils
os.environ['DEBUG'] = '1'
os.environ['SHOW_LLM_INPUT_MSG'] = '1'
wf_check_db_structure = workflows.check_db_structure.clone()
t_idx = 71
q_idx = 0
question_team = config.all_question[t_idx]
question_item = question_team["team"][q_idx]
show(question_item["id"])
show(question_item["rewrited_question"])
facts = []
qas = []
sql_results = []
for idx in range(q_idx):
    qas.append([
        {"role": "user", "content": utils.ajust_org_question(question_team["team"][idx]["question"])},
        {"role": "assistant", "content": question_team["team"][idx]["answer"]},
    ])
    if "sql_results" in question_team["team"][idx]:
        sql_results.append(copy.deepcopy(question_team["team"][idx]["sql_results"]))
    else:
        sql_results.append([])
if "facts" in question_item:
    facts = copy.deepcopy(question_item["facts"])
# 注入已知事实
key_facts = "已知信息"
if len(facts) > 0:
    kv = {key_facts: "\n".join(facts)}
    wf_check_db_structure.agent_column_selector.add_system_prompt_kv(kv)
# 注入历史对话
qas_content = [
    f"Question: {qa['content']}" if qa["role"] == "user" else f"Answer: {qa['content']}"
    for qa in qas
]
key_qas = "历史对话"
if len(qas_content) > 0:
    val_qas = ""
    for qa_idx, qa_content in enumerate(qas_content):
        if qa_idx > 0:
            val_qas += "---\n"
        val_qas += f"{qa_content}\n"
        if sql_results[qa_idx] != []:
            val_qas += (
                "用到以下SQL查询（供后续问答理解本问题的答案如何得来，后续问答可参考）：\n" +
                "\n".join(sql_results[qa_idx])
            )
    kv = {key_qas: val_qas}
    wf_check_db_structure.agent_column_selector.add_system_prompt_kv(kv)
else:
    wf_check_db_structure.agent_column_selector.del_system_prompt_kv(key_qas)
res = wf_check_db_structure.run(inputs={"messages":[
    {"role": "user", "content": question_item["rewrited_question"]}
]})
show(res)

tttt----72----22-3-1
What is the full English name of Huazhu Group Ltd. as listed on the U.S. stock market? (Please provide the company's full name)


>>>>> 【user】 Said:
提问:
What is the full English name of Huazhu Group Ltd. as listed on the U.S. stock market? (Please provide the company's full name)


>>>>> Agent【decode_question】 Said:
What is the full English name of Huazhu Group Ltd.
Which stock market is Huazhu Group Ltd. listed on
What is the full name of the company as listed on the U.S. stock market

get_relevant_table_columns --- 向量搜索 ---
question: What is the full English name of Huazhu Group Ltd.
constantdb.hk_secumain: 0.441 (最高分: 0.468, 总分: 6.733, 字段数: 19, 
最佳列: [('EngNameAbbr', 0.46769786656914586), ('EngName', 0.45615033966253987), ('FormerName', 0.44265947467552863), ('ChiName', 0.4121418067133752), ('SecuCode', 0.3708765625527573), ('ListedSector', 0.35907801855608956), ('SecuMarket', 0.35188391443205325), ('SecuCategory', 0.3490748620407707), ('ChiSpelling', 0.34873146

### 重写问题

In [1]:
import os
import agents
from src.utils import show
os.environ['DEBUG'] = '1'
os.environ['SHOW_LLM_INPUT_MSG'] = '1'

msg = """
下面是顺序的历史问答:
'''
Question: 天士力在2020年的最大担保金额是多少？答案需要包含1位小数
Answer: 天士力在2020年的最大担保金额是1620000000.0元。
Question: 天士力在2020年的最大担保金额涉及的担保方是谁？（请回答公司全称）担保金额是多少（答案需要包含2位小数）？
Answer: 担保方: 天士力医药集团股份有限公司, 金额: 1620000000.00元
'''
现在用户继续提问，请根据已知信息，理解当前这个问题的完整含义，并重写这个问题使得单独拿出来看仍然能够正确理解。用户的问题是：
天士力在2020年最新的担保事件是什么？答案包括事件内容、担保方（公司全称）、被担保方（公司全称）、担保金额（答案需要包1位小数）和日期信息（格式为YYYY年MM月DD日）。
"""

agents.agent_rewrite_question.clear_history()
new_question, _ = agents.agent_rewrite_question.answer(msg)

已从 /data/workspace/howard/jinglever/competition/2024-FinGLM2-semi-final/cache/table_relations.json 加载表关系图


>>>>> 【user】 Said:

下面是顺序的历史问答:
'''
Question: 天士力在2020年的最大担保金额是多少？答案需要包含1位小数
Answer: 天士力在2020年的最大担保金额是1620000000.0元。
Question: 天士力在2020年的最大担保金额涉及的担保方是谁？（请回答公司全称）担保金额是多少（答案需要包含2位小数）？
Answer: 担保方: 天士力医药集团股份有限公司, 金额: 1620000000.00元
'''
现在用户继续提问，请根据已知信息，理解当前这个问题的完整含义，并重写这个问题使得单独拿出来看仍然能够正确理解。用户的问题是：
天士力在2020年最新的担保事件是什么？答案包括事件内容、担保方（公司全称）、被担保方（公司全称）、担保金额（答案需要包1位小数）和日期信息（格式为YYYY年MM月DD日）。



>>>>> Agent【rewrite_question】 Said:
天士力在2020年最新的担保事件包括哪些内容？请提供事件内容、担保方（天士力医药集团股份有限公司）、被担保方（公司全称）、担保金额（1620000000.0元）和具体日期信息（格式为YYYY年MM月DD日）。


### 基于向量搜索

In [1]:
import os
import config, workflows
import utils
from src.utils import show
os.environ['DEBUG'] = '1'
os.environ['SHOW_LLM_INPUT_MSG'] = '1'
# t_idx = 0
# q_idx = 0
# question_team = config.all_question[t_idx]
# question_item = question_team["team"][q_idx]
# show(question_item["rewrited_question"])
# quetion = "提问:\n" + question_item["rewrited_question"]
# ag_decode_question = workflows.check_db_structure.agent_decode_question.clone()
# answer, cnt = ag_decode_question.answer(quetion)
# show(cnt)
# question_list = [q.strip() for q in answer.split("\n") if q.strip() != ""]
question_list = [
"2021年1月的交易日有哪些",
"各一级行业在2021年1月的股价波动性如何计算",
"2021年1月股价波动性最高的三个一级行业是哪些",
]

ts = utils.get_relevant_table_columns(question_list)


已从 /data/workspace/howard/jinglever/competition/2024-FinGLM2-semi-final/cache/table_relations.json 加载表关系图

get_relevant_table_columns --- 向量搜索 ---
question: 2021年1月的交易日有哪些
constantdb.qt_tradingdaynew: 0.620 (最高分: 0.680, 总分: 3.778, 字段数: 7, 
最佳列: [('TradingDate', 0.6797088988889104), ('IfTradingDay', 0.6568310780894282), ('IfMonthEnd', 0.5122853705948297), ('IfQuarterEnd', 0.5098794690163779), ('IfWeekEnd', 0.5076235352020274), ('IfYearEnd', 0.494175871687218), ('SecuMarket', 0.41757749330793503)])
astockmarketquotesdb.qt_dailyquote: 0.607 (最高分: 0.709, 总分: 4.961, 字段数: 10, 
最佳列: [('TradingDay', 0.708895924022065), ('TurnoverVolume', 0.5325221508857225), ('TurnoverDeals', 0.5289600546879775), ('ClosePrice', 0.526991712903516), ('TurnoverValue', 0.5026111734164944), ('LowPrice', 0.4724830909605653), ('PrevClosePrice', 0.4681741610296136), ('HighPrice', 0.4482290182413438), ('InnerCode', 0.38889137570236054), ('OpenPrice', 0.38373175164813744)])
usstockdb.us_dailyquote: 0.535 (最高分: 0.590, 总分

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


question: 2021年1月股价波动性最高的三个一级行业是哪些
astockindustrydb.lc_indfinindicators: 0.521 (最高分: 0.546, 总分: 27.210, 字段数: 61, 
最佳列: [('Standard', 0.54577899393652), ('IndTotalShares', 0.5331843851813265), ('DebtAssetsRatio', 0.5313017344287694), ('CurrentRatio', 0.5113018870618846), ('IndNPCOwnersTTM', 0.4998816712673614), ('DilutedROA', 0.49970234032487326), ('NetOperCFToToOperReve', 0.49656223744701355), ('PeriodCostsRate', 0.4943297983278542), ('ROEAvg', 0.49129882109479717), ('ROECut', 0.49092181740600604), ('NetProfitRatioTTM', 0.4884265094675273), ('IndOperatingProfit', 0.4852701576056404), ('IndustryCode', 0.48521291366982255), ('GrossProfitYOY', 0.480458497681393), ('QuickRatio', 0.4800382395362411)])
astockindustrydb.lc_industryvaluation: 0.515 (最高分: 0.558, 总分: 8.776, 字段数: 20, 
最佳列: [('NegotiableMV', 0.5575317455814672), ('FreeFloatMV', 0.5135332760076983), ('PE_LYR', 0.48644300273511143), ('TradingDay', 0.48010022437288613), ('IndustryCode', 0.46551135598415816), ('PE_TTM', 0.460033384530

Loading model cost 0.591 seconds.
Prefix dict has been built successfully.


question: 2021年1月的交易日有哪些
constantdb.lc_areacode: 16.174 (最高分: 18.931, 总分: 96.086, 字段数: 13, 
最佳列: [('AreaEngNameAbbr', 18.93147695754486), ('ChangeNote', 17.19081368937041), ('ParentName', 16.177522937655834), ('CancelDate', 16.079572828492864), ('IfEffected', 15.986249886519907), ('FirstLevelCode', 4.091691787300718), ('AreaChiName', 4.074869794288489), ('ParentNode', 3.554071902177613), ('AreaEngName', 0.0), ('SecondLevelCode', 0.0), ('AreaCode', 0.0), ('AreaInnerCode', 0.0), ('Remark', 0.0)])
question: 各一级行业在2021年1月的股价波动性如何计算
astockindustrydb.lc_indfinindicators: 15.312 (最高分: 16.711, 总分: 620.716, 字段数: 61, 
最佳列: [('ARTRate', 16.710929609953837), ('GrossProfitYOY', 16.27659792381789), ('NetProfitRatioCut', 16.229735426531118), ('IndTotalAssets', 16.144954962983675), ('ROETTM', 13.719649414273723), ('ROEAvg', 13.42831538812826), ('NetOperCFToToOperReve', 13.355850097000854), ('IndNetProfitTTM', 13.316619859905828), ('WROECut', 13.266177644464946), ('OperatingExpenseRate', 13.19778617088

### 基于词频搜索

In [2]:
import os
import config, workflows
import utils
from src.utils import show
os.environ['DEBUG'] = '1'
os.environ['SHOW_LLM_INPUT_MSG'] = '1'
t_idx = 0
q_idx = 0
top_p = 1
save_column_num = 8

question_team = config.all_question[t_idx]
question_item = question_team["team"][q_idx]
show(question_item["rewrited_question"])
quetion = "提问:\n" + question_item["rewrited_question"]
ag_decode_question = workflows.check_db_structure.agent_decode_question.clone()
answer, cnt = ag_decode_question.answer(quetion)
show(cnt)
question_list = [q.strip() for q in answer.split("\n") if q.strip() != ""]

for question in question_list:
    print("\nQuery:", question)
    ts = utils.calculate_table_tf_idf_score(question, top_p)
    for t in ts:
        print(f"{t[0]}: {t[1]:.3f} (最高分: {t[2]:.3f}, 总分: {t[3]:.3f}, 字段数: {t[4]}, \n最佳列: {t[5][:save_column_num]})")


天士力在2020年的最大担保金额是多少？请提供包含1位小数的答案。


>>>>> 【user】 Said:
提问:
天士力在2020年的最大担保金额是多少？请提供包含1位小数的答案。


>>>>> Agent【decode_question】 Said:
天士力在2020年的担保明细有哪些
天士力在2020年的最大担保金额是多少
如何将最大担保金额保留1位小数

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache



727

Query: 天士力在2020年的担保明细有哪些


Loading model cost 0.513 seconds.
Prefix dict has been built successfully.


astockeventsdb.lc_warrant: 16.482 (最高分: 18.021, 总分: 527.358, 字段数: 47, 
最佳列: [('SecuredPartyAssociation', 18.021442140297015), ('EventContent', 17.30779925291839), ('ObjectName', 16.942617675916594), ('CGuarantorAssociation', 15.65193443034981), ('CGuaranteeAsset', 15.607551887300387), ('LendBeginDate', 15.41255449895975), ('DischargeGuaranteeWays', 15.297500574440079), ('SecuredParty', 15.235183417570944)])

Query: 天士力在2020年的最大担保金额是多少
astockeventsdb.lc_warrant: 15.047 (最高分: 17.173, 总分: 394.854, 字段数: 47, 
最佳列: [('GuaranteeReason', 17.17345581076443), ('GuarantorCompany', 16.161788933435453), ('SubjectAssociation', 14.461731806216585), ('ObjectCode', 12.496584548130006), ('AgreementDate', 12.329474418643235), ('DischargeGuaranteeWays', 12.231023248681076), ('SecuredParty', 11.341658717963067), ('ObjectName', 10.899201225702917)])

Query: 如何将最大担保金额保留1位小数
astockeventsdb.lc_warrant: 12.528 (最高分: 15.065, 总分: 310.896, 字段数: 47, 
最佳列: [('GuaranteeReason', 15.065215941021155), ('SubjectAssociati

### 查最短外链路径

In [4]:
import config
import json
print(
    # config.table_relations.print_all_paths(
    json.dumps(
        config.table_relations.find_shortest_path(
            "astockmarketquotesdb.qt_dailyquote",
            "astockmarketquotesdb.qt_stockperformance"
        ),
        indent=4,
        ensure_ascii=False
    )
    # )
)

[
    [
        "astockmarketquotesdb.qt_dailyquote",
        "constantdb.secumain",
        [
            "证券内部编码",
            "InnerCode-InnerCode"
        ]
    ],
    [
        "constantdb.secumain",
        "astockmarketquotesdb.qt_stockperformance",
        [
            "证券内部编码",
            "InnerCode-InnerCode"
        ]
    ]
]


### 拆解子问题

In [1]:
import os, config, workflows
from src.utils import show
os.environ['DEBUG'] = '1'
os.environ['SHOW_LLM_INPUT_MSG'] = '1'

t_idx = 92
q_idx = 0
question_team = config.all_question[t_idx]
question_item = question_team["team"][q_idx]
show(question_item["rewrited_question"])
quetion = "提问:\n" + question_item["rewrited_question"]
ag_decode_question = workflows.check_db_structure.agent_decode_question.clone()
answer, cnt = ag_decode_question.answer(quetion)
show(cnt)

question_list = [q.strip() for q in answer.split("\n") if q.strip() != ""]

已从 /data/workspace/howard/jinglever/competition/2024-FinGLM2-semi-final/cache/table_relations.json 加载表关系图
华峰化学在2019年至2021年期间的PBX值是多少？请提供包含两位小数的答案。


>>>>> 【user】 Said:
提问:
华峰化学在2019年至2021年期间的PBX值是多少？请提供包含两位小数的答案。


>>>>> Agent【decode_question】 Said:
华峰化学在2019年至2021年期间的PBX值是多少
PBX值的计算公式是什么
如何从相关数据表中提取华峰化学2019年至2021年的PBX值所需数据
如何确保PBX值计算结果包含两位小数
813


### 格式化sql

In [2]:
import sqlparse, re
sql = '''
SELECT SUM(ActualCashDiviRMB) AS TotalDividend
FROM astockfinancedb.lc_dividend
WHERE InnerCode = 12972
AND DATE(EndDate) BETWEEN '2020-01-01' AND '2020-12-31';
'''
sql = sqlparse.format(sql, reindent=False, keyword_case='upper', strip_comments=True)
sql = re.sub(r'\s+', ' ', sql).strip()
print(sql)


SELECT SUM(ActualCashDiviRMB) AS TotalDividend FROM astockfinancedb.lc_dividend WHERE InnerCode = 12972 AND DATE(EndDate) BETWEEN '2020-01-01' AND '2020-12-31';


In [2]:
# question = "机器人2020年发布的19年年报的大股东是谁？"
# agents.agent_extract_company.clear_history()
# answer, _ = agents.agent_extract_company.answer((
#     # '''提取下面这段文字里的实体，如公司名、股票代码等，如果识别结果是空，那么就回复No Entities.'''
#     '''提取下面这段文字中的实体（如公司名、股票代码、拼音缩写等），如果识别结果是空，那么就回复No Entities.'''
#     f'''"{question}"'''
# ))

In [3]:
# show(utils.query_company("GLYY"))

In [4]:
# enum_columns = {}
# for t_name, table in config.table_column.items():
#     filtered_columns = {col['column']: col['desc'] for col in table if '具体描述' in col['desc']}
#     if filtered_columns:
#         enum_columns[t_name] = filtered_columns
# show(enum_columns)

In [5]:
# agents.agent_rewrite_question.clear_history()
# facts = []
# qas_content = [] 
# question = "GLYY中文全称是什么？"
# new_question, _ = agents.agent_rewrite_question.answer(
#     ("历史问答:无。\n" if len(qas_content) == 0 else "下面是顺序的历史问答:\n'''\n" + "\n".join(qas_content) + "\n'''\n") +
#     f"现在用户继续提问，请根据已知信息，理解当前这个问题的完整含义，并重写这个问题使得单独拿出来看仍然能够正确理解：{question}"
# )
# show(new_question)



>>>>> 【user】 Said:
历史问答:无。
现在用户继续提问，请根据已知信息，理解当前这个问题的完整含义，并重写这个问题使得单独拿出来看仍然能够正确理解：GLYY中文全称是什么？


>>>>> Agent【rewrite_question】 Said:
请问GLYY的中文全称是什么？
请问GLYY的中文全称是什么？
